In [1]:
!pip -q install underthesea mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 84.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 67.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 99.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 8.9 MB/s eta 0:00:00


In [14]:
import numpy as np
import tensorflow as tf
import torch

In [3]:
!pip install pyvi
!pip install tqdm
!pip install keras
!pip install -q datasets
!pip -q install underthesea mlflow

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 52.7 MB/s eta 0:00:0000:0100:01


In [4]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout, Concatenate
from keras.layers import LSTM, Embedding, Bidirectional, GRU
from keras.layers import SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.initializers import Constant
from keras.preprocessing.sequence import pad_sequences
#from keras.preprocessing.text import Tokenizer
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy
import pickle
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import collections
from sklearn.utils.class_weight import compute_class_weight
from gensim import models
import torch
from transformers import AutoModel, AutoTokenizer
from pyvi.ViTokenizer import tokenize
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from pyarrow.lib import _Weakrefable
import pyarrow
# from datasets import load_dataset

In [6]:
def replace_values(value):
    if value == 0:
        return np.nan  # Thay 0 thành null (NaN)
    elif value == 1:
        return 0
    elif value == 2:
        return 1
    elif value == 3:
        return 2
    return value


data_train = pd.read_csv('/kaggle/input/sentimentabsa/data_train.csv')
data_val = pd.read_csv('/kaggle/input/sentimentabsa/data_val.csv')
data_test = pd.read_csv('/kaggle/input/sentimentabsa/data_test.csv')

# Áp dụng hàm thay thế cho các cột A, B, C, D
columns_to_replace = ['Quality', 'Smell', 'Shipping', 'Packing', 'Price', 'Other', 'General']
data_train[columns_to_replace] = data_train[columns_to_replace].applymap(replace_values)
data_test[columns_to_replace] = data_test[columns_to_replace].applymap(replace_values)
data_val[columns_to_replace] = data_val[columns_to_replace].applymap(replace_values)


data_train.drop('Unnamed: 0', axis = 1, inplace = True)
data_val.drop('Unnamed: 0', axis = 1, inplace = True)
data_test.drop('Unnamed: 0', axis = 1, inplace = True)
data_test.rename(columns={'Tokenized': 'processed_content'}, inplace=True)
data_train.rename(columns={'Tokenized': 'processed_content'}, inplace=True)
data_val.rename(columns={'Tokenized': 'processed_content'}, inplace=True)
data_test.rename(columns={'Quality': 'quality'}, inplace=True)
data_train.rename(columns={'Quality': 'quality'}, inplace=True)
data_val.rename(columns={'Quality': 'quality'}, inplace=True)
data_train.rename(columns={'Smell': 'smell'}, inplace=True)
data_test.rename(columns={'Smell': 'smell'}, inplace=True)
data_val.rename(columns={'Smell': 'smell'}, inplace=True)
data_val.rename(columns={'Shipping': 'shipping'}, inplace=True)
data_train.rename(columns={'Shipping': 'shipping'}, inplace=True)
data_test.rename(columns={'Shipping': 'shipping'}, inplace=True)
data_test.rename(columns={'Packing': 'packing'}, inplace=True)
data_train.rename(columns={'Packing': 'packing'}, inplace=True)
data_val.rename(columns={'Packing': 'packing'}, inplace=True)
data_train.rename(columns={'Price': 'price'}, inplace=True)
data_val.rename(columns={'Price': 'price'}, inplace=True)
data_test.rename(columns={'Price': 'price'}, inplace=True)
data_val.rename(columns={'Other': 'others'}, inplace=True)
data_test.rename(columns={'Other': 'others'}, inplace=True)
data_train.rename(columns={'Other': 'others'}, inplace=True)

<ipython-input-6-1b4dc075d9a1>:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_train[columns_to_replace] = data_train[columns_to_replace].applymap(replace_values)
<ipython-input-6-1b4dc075d9a1>:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_test[columns_to_replace] = data_test[columns_to_replace].applymap(replace_values)
<ipython-input-6-1b4dc075d9a1>:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_val[columns_to_replace] = data_val[columns_to_replace].applymap(replace_values)


In [10]:
def make_outputs(df):
    outputs = []
    max_columns = len(df.columns) - 2  # Fixed number of columns
    for row in range(len(df)):
        row_one_hot = []
        for col in range(1, min(max_columns + 1, len(df.columns)-1)):
            sentiment = df.iloc[row, col]
            if pd.isna(sentiment):
                one_hot = [0, 0, 0, 0]  # Default for NaN
            elif sentiment == 0: one_hot = [1, 0, 0, 0]
            elif sentiment == 1: one_hot = [0, 1, 0, 0]
            elif sentiment == 2: one_hot = [0, 0, 1, 0]
            elif sentiment == 3: one_hot = [0, 0, 0, 1]
            else:
                raise ValueError(f"Unexpected sentiment value {sentiment} at row {row}, column {col}")
            row_one_hot.append(one_hot)
        outputs.append(row_one_hot)
    return np.array(outputs, dtype='uint8')

# Example usage
label_train = make_outputs(data_train)
label_val = make_outputs(data_val)
label_test = make_outputs(data_test)


In [11]:
label_train = make_outputs(data_train)
label_val = make_outputs(data_val)
label_test = make_outputs(data_test)

print('Train outputs:', label_train.shape)
print('Validate outputs:', label_val.shape)
print('Test outputs:', label_test.shape)

Train outputs: (10198, 7, 4)
Validate outputs: (2186, 7, 4)
Test outputs: (2185, 7, 4)


In [15]:
from underthesea import word_tokenize
period_strip = re.compile(r'(?!<=\d)(\.)(?!\d)')
comma_strip = re.compile(r'(\d)(,)(\d)')
punctuation_chars = re.escape(r';/[]"{}()=+\_-><@`,?!.')
punctuation = re.compile(r'([{}])'.format(re.escape(punctuation_chars)))
punctuation_with_a_space = re.compile(r'(?<= )([{0}])|([{0}])(?= )'.format(punctuation_chars))

def process_punctuation(s):
    if punctuation.search(s) is None:
        return s
    s = punctuation_with_a_space.sub('', s)
    if re.search(comma_strip, s) is not None:
        s = s.replace(',', '')
    s = punctuation.sub(' ', s)
    s = period_strip.sub('', s)
    return s.strip()

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

def encode_data(df):
    return tokenizer(data_train['processed_content'].apply(lambda x: word_tokenize(process_punctuation(x.lower()), format="text")).tolist(), padding="max_length", truncation=True, max_length=256, return_tensors="tf")

train_encodings = encode_data(data_train)
val_encodings = encode_data(data_val)
test_encodings = encode_data(data_test)

In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [18]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer
import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import Dataset, DataLoader

# Tải mô hình và tokenizer
model_name = "vinai/phobert-base-v2"
num_labels = 7

from transformers import RobertaForSequenceClassification, RobertaConfig, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

texts_train = data_train['processed_content'].apply(lambda x: word_tokenize(process_punctuation(x.lower()), format="text")).values
labels_train = label_train

texts_val = data_val['processed_content'].apply(lambda x: word_tokenize(process_punctuation(x.lower()), format="text")).values
labels_val = label_val
from transformers import TrainerCallback

class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience=3):
        self.early_stopping_patience = early_stopping_patience
        self.early_stopping_counter = 0
        self.best_loss = float('inf')

    def on_evaluate(self, args, state, control, **kwargs):
        eval_loss = kwargs.get("metrics", {}).get("eval_loss", None)
        if eval_loss is not None:
            if eval_loss < self.best_loss:
                self.best_loss = eval_loss
                self.early_stopping_counter = 0
            else:
                self.early_stopping_counter += 1
            if self.early_stopping_counter >= self.early_stopping_patience:
                control.should_training_stop = True
                print(f"Early stopping triggered: No improvement in the last {self.early_stopping_patience} evaluations.")

class MultiLabelDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, item):
        text = self.texts[item]
        labels = self.labels[item]
        label_tensor = torch.tensor(labels, dtype=torch.float32).view(-1)
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label_tensor
        }

dataset_train = MultiLabelDataset(texts_train, labels_train, tokenizer)
dataset_val = MultiLabelDataset(texts_val, labels_val, tokenizer)
model = RobertaForSequenceClassification.from_pretrained("vinai/phobert-base-v2", num_labels=28)
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=1e-5,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",  # evaluation_strategy
    report_to="none"        # W&B
)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = BCEWithLogitsLoss()
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

    def predict(self, test_dataset):
        predictions, label_ids, metrics = super().predict(test_dataset)
        probs = torch.sigmoid(torch.tensor(predictions))
        preds = (probs).int()
        return preds, label_ids, metrics

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
trainer.evaluate()

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.441400,0.415058
2,0.180000,0.170201
3,0.125400,0.120896
4,0.111200,0.107615
5,0.104700,0.099845
6,0.095800,0.093339
7,0.092800,0.088773
8,0.084200,0.083714
9,0.086000,0.081069
10,0.080600,0.080229


{'eval_loss': 0.08022899925708771,
 'eval_runtime': 18.7105,
 'eval_samples_per_second': 116.833,
 'eval_steps_per_second': 2.94,
 'epoch': 10.0}

In [19]:
model_path = '/kaggle/working/'
model.save_pretrained(model_path)

In [29]:
texts_test = data_test['processed_content'].apply(lambda x: word_tokenize(process_punctuation(x.lower()), format="text")).values
labels_test = label_test
dataset_test = MultiLabelDataset(texts_test, labels_test, tokenizer)
prediction = trainer.predict(dataset_test)
prediction

(tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 {'test_loss': 0.08034634590148926,
  'test_runtime': 17.5883,
  'test_samples_per_second': 124.23,
  'test_steps_per_second': 3.127})

In [30]:
pred = prediction
labels = pred[1].reshape(len( pred[1]), 7, 4)
preds = pred[0].reshape(len( pred[1]), 7, 4)

In [31]:
aspect_labels = 1 - labels[:, :, 0]
aspect_preds = 1 - preds[:, :, 0]

In [32]:
# aspect detection
from sklearn.metrics import f1_score
aspect_f1 = f1_score(aspect_labels.flatten(), aspect_preds.flatten())
aspect_f1

0.9951052856345061

In [33]:
# Sentiment
sentiment_labels = np.argmax(labels[:, : , 1:], axis = -1)
sentiment_preds = np.argmax(preds[: , : , 1:], axis = -1)

In [34]:
sentiment_labels = np.argmax(labels[:, : , 1:], axis = -1)
sentiment_preds = np.argmax(preds[: , : , 1:], axis = -1)
sentiment_f1 = f1_score(sentiment_labels.flatten(), sentiment_preds.flatten(), average = 'weighted')
sentiment_f1

0.8182459831480563

In [35]:
from sklearn.metrics import f1_score

# Đảm bảo chuyển tensor về CPU và numpy array nếu cần
negative_preds = preds[:, :, 1].cpu().numpy().flatten() if torch.is_tensor(preds) else preds[:, :, 1].flatten()
negative_labels = labels[:, :, 1].cpu().numpy().flatten() if torch.is_tensor(labels) else labels[:, :, 1].flatten()

neutral_preds = preds[:, :, 2].cpu().numpy().flatten() if torch.is_tensor(preds) else preds[:, :, 2].flatten()
neutral_labels = labels[:, :, 2].cpu().numpy().flatten() if torch.is_tensor(labels) else labels[:, :, 2].flatten()

positive_preds = preds[:, :, 3].cpu().numpy().flatten() if torch.is_tensor(preds) else preds[:, :, 3].flatten()
positive_labels = labels[:, :, 3].cpu().numpy().flatten() if torch.is_tensor(labels) else labels[:, :, 3].flatten()

# Tính F1-score
f1_negative = f1_score(negative_labels, negative_preds)
f1_neutral = f1_score(neutral_labels, neutral_preds)
f1_positive = f1_score(positive_labels, positive_preds)

print("F1 Score - Negative:", f1_negative)
print("F1 Score - Neutral:", f1_neutral)
print("F1 Score - Positive:", f1_positive)


F1 Score - Negative: 0.0
F1 Score - Neutral: 0.0
F1 Score - Positive: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [36]:
y_pred = preds

y_true = labels

# Xử lý và chuyển đổi sang nhãn tương ứng
sentiment_labels = np.argmax(y_true[:, :, 1:], axis=1)
sentiment_preds = np.argmax(y_pred[:,:, 1:], axis=1)

# Cách 1: Tính F1 score với average='macro'
f1_macro = f1_score(sentiment_labels.flatten(), sentiment_preds.flatten(), average='macro')

# Cách 2: Tính F1 cho từng sentiment riêng lẻ
negative_preds = y_pred[:,:, 1]
negative_labels = y_true[:,:, 1]
neutral_preds = y_pred[:, :,2]
neutral_labels = y_true[:, :,2]
positive_preds = y_pred[:, :,3]
positive_labels = y_true[:, :,3]

f1_negative = f1_score(negative_labels.flatten(), negative_preds.flatten())
f1_neutral = f1_score(neutral_labels.flatten(), neutral_preds.flatten())
f1_positive = f1_score(positive_labels.flatten(), positive_preds.flatten())

# Lấy trung bình của các F1 scores
f1_average = np.mean([f1_negative, f1_neutral, f1_positive])

# In kết quả
print("F1 Score (Macro Average):", f1_macro)
print("F1 Scores Individually: Negative = {}, Neutral = {}, Positive = {}".format(f1_negative, f1_neutral, f1_positive))
print("F1 Score (Average of Individual Scores):", f1_average)

F1 Score (Macro Average): 0.12745358410391366
F1 Scores Individually: Negative = 0.0, Neutral = 0.0, Positive = 0.0
F1 Score (Average of Individual Scores): 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [38]:
from sklearn.metrics import classification_report
import numpy as np

for aspect_index in range(5):
    print(f"Aspect {aspect_index + 1}")
    aspect_true = np.argmax(labels[:, aspect_index, :], axis=1)
    aspect_pred = np.argmax(preds[:, aspect_index, :], axis=1)
    
    # Get the unique classes in aspect_true and aspect_pred
    unique_classes = np.unique(np.concatenate((aspect_true, aspect_pred)))
    
    # Map unique_classes to the corresponding target_names
    all_target_names = ["None", "Negative", "Neutral", "Positive"]
    target_names = [all_target_names[i] for i in unique_classes]
    
    print(classification_report(aspect_true, aspect_pred, target_names=target_names, zero_division=0))


Aspect 1
              precision    recall  f1-score   support

        None       0.67      1.00      0.81      1472
    Negative       0.00      0.00      0.00        62
     Neutral       0.00      0.00      0.00       651

    accuracy                           0.67      2185
   macro avg       0.22      0.33      0.27      2185
weighted avg       0.45      0.67      0.54      2185

Aspect 2
              precision    recall  f1-score   support

        None       0.98      1.00      0.99      2151
    Negative       0.00      0.00      0.00        21
     Neutral       0.00      0.00      0.00        13

    accuracy                           0.98      2185
   macro avg       0.33      0.33      0.33      2185
weighted avg       0.97      0.98      0.98      2185

Aspect 3
              precision    recall  f1-score   support

        None       0.91      1.00      0.95      1994
    Negative       0.00      0.00      0.00         3
     Neutral       0.00      0.00      0.00     